# Federated Learning with Homomorphic Encryption for Secure Aggregation

Please make sure you set up a virtual environment and follow [example root readme](../../README.md) before starting this notebook.
Then, install the requirements.

<div class="alert alert-block alert-info"> <b>NOTE</b> Some of the cells below generate long text output.  We're using <pre>%%capture --no-display --no-stderr cell_output</pre> to suppress this output.  Comment or delete this line in the cells below to restore full output.</div>

In [ ]:
%%capture --no-display --no-stderr cell_output
import sys
!{sys.executable} -m pip install -r requirements.txt

### Homomorphic Encryption in NVFlare
This example includes instructions on running [FedAvg](https://arxiv.org/abs/1602.05629) with [homomorphic encryption](https://developer.nvidia.com/blog/federated-learning-with-homomorphic-encryption/) (originally developed for Clara Train's FL feature, the predecessor of NVFlare).

It uses the provisioning and the admin API to submit jobs, similar to how one would set up experiments in real-world deployment.

HE encryption and decryption are added as filters using the [FedJob API](https://nvflare.readthedocs.io/en/main/programming_guide/fed_job_api.html#fedjob-api) you should have seen in prior chapters.

The communication channels in NVFlare are TLS-encrypted using the certificates provided by the provisioning tool. However, each client can receive additional keys to homomorphically encrypt their model updates before sending them to the server. The server doesn’t own a key and only sees the encrypted model updates.

HE allows the server to aggregate these encrypted weights and then send the updated model back to the client. The clients can decrypt the model weights because they have the keys and can continue with the next round of training.

![Homomorphic Encryption](he.png)

# SAG workflow with Homomorphic Encryption

In this example, we will demonstrate how to use homomorphic encryption (HE)
by building on the previous [FedAvg with SAG workflow](../sag/sag.ipynb#title) example using the CIFAR10 dataset.

## Homomorphic Encryption

Homomorphic encryption is a form of encryption that allows computations to be performed on encrypted data, which is especially useful for preserving privacy in a federated learning setting.
FLARE uses the [TenSEAL](https://github.com/OpenMined/TenSEAL) library to enable homomorphic encryption operations on tensors in the provided FLARE [HE implementation](https://github.com/NVIDIA/NVFlare/tree/main/nvflare/app_opt/he).

## Job Configuration

On the client side, to configure homomorphic encryption we must add the following HE filters in config_fed_client.conf:

- `HEModelDecryptor`: Data filter to decrypt incoming Shareable objects using HE.
- `HEModelEncryptor`: Result filter to encrypt outgoing Shareable objects using HE.

On the server side, we configure components with HE support in config_fed_server.conf:

- `HEModelShareableGenerator`: This ShareableGenerator converts between Shareable and Learnable objects using HE.
- `HEInTimeAccumulateWeightedAggregator`: Filter to encrypt Shareable object using HE.
- `HEModelSerializeFilter`: Used to serialize TenSEAL encrypted server models for use with HE support.

Let's use the Job CLI to create the job from the sag_pt_he template:

In [ ]:
! nvflare config -jt ../../../../../../job_templates

In [ ]:
! nvflare job create -j /tmp/nvflare/jobs/cifar10_sag_pt_he -w sag_pt_he -sd code/fl -force

We can take a look at the client and server configurations and make any changes as desired:

In [ ]:
! cat /tmp/nvflare/jobs/cifar10_sag_pt_he/app/config/config_fed_client.conf

In [ ]:
! cat /tmp/nvflare/jobs/cifar10_sag_pt_he/app/config/config_fed_server.conf

## Prepare Data

Make sure the CIFAR10 dataset is downloaded with the following script:

In [ ]:
! python data/download.py --dataset_path /tmp/nvflare/data/cifar10

## Run the Job using POC mode

As the use of HE is intended for real-world deployment, we cannot directly use the FL simulator, but we can use the POC (proof-of-concept) mode for local deployment.

To support HE, we need the provisioning process to generate and write the TenSEAL homomorphic encryption contexts for the server and client.
Currently the simulator does not support HE for secure aggregation, however we can use the POC command `-he` option to prepare the HE supported POC workspace with the `HEBuilder`:

In [ ]:
! nvflare poc clean
! nvflare poc prepare -he

Start the FLARE system in POC mode in another terminal with ```nvflare poc start -ex admin@nvidia.com```

Finally we can submit our HE configured job:

In [ ]:
! nvflare job submit -j /tmp/nvflare/jobs/cifar10_sag_pt_he

Once you are finished with the example, you can shutdown and clean the POC system:

In [ ]:
! nvflare poc stop

In [ ]:
! nvflare poc clean

As an additional resource, see the [CIFAR10 Real World Example](https://github.com/NVIDIA/NVFlare/tree/main/examples/advanced/cifar10/cifar10-real-world) for creating a secure workspace for HE using provisioning instead of POC mode.

Next, we will learn about secure Kaplan Meier survival analysis also utilizing [homomorphic encryption](05.3.2_kaplan_meier_survival_analysis_with_he.ipynb).